In [1]:
import requests
from urllib import parse

import pandas as pd
import numpy as np

import time
from tqdm import tqdm

import warnings
warnings.filterwarnings('ignore')

In [2]:
api_key = 'Sxwiysbra37brnJK9Fp12Z81vXvU0IEN'

In [3]:
#경매장 시세 검색

def df_aution_sold(itemName, limit=100):
    url = f'https://api.neople.co.kr/df/auction-sold'
    params = {
        'itemName' : itemName,
        'limit' : limit,
        'apikey' : api_key
    }
    
    return requests.get(url, params=params)

def df_search_item(itemName = '', item_hashtag = ''):
    url = 'https://api.neople.co.kr/df/items'

    params = {
        'limit' : 30,
        'apikey' : api_key,
        'wordType' : 'full'
    }

    if itemName != '':
        params['itemName'] = itemName
    if item_hashtag != '':
        params['hashtag'] = item_hashtag

    return requests.get(url, params=params)

def df_aution_item(itemName, limit=100):
    url = f'https://api.neople.co.kr/df/auction'
    params = {
        'itemName' : itemName,
        'limit' : limit,
        'sort' : 'auctionNo:desc',
        'apikey' : api_key
        
    }
    
    return requests.get(url, params=params)

def check_new_items(new_rows, df):
    if len(df) == 0:
        return new_rows.index
    else:
        lastNo = df['auctionNo'].max()
        
    new_items = new_rows[new_rows['auctionNo'] > lastNo]
    
    if len(new_items):
        return new_items.index
    else:
        return []
    

def check_sold_item(new_rows, df):
    new_list = new_rows['auctionNo'].values

    sold_items_idx = df[~df['auctionNo'].isin(new_list)].index

    return sold_items_idx

def df_get_auction_reg_item(df,target, saled_check_list):
    return_check_list = saled_check_list.copy()
    if len(df) == 0:
        new_data = pd.DataFrame(df_aution_item(target).json()['rows'])
        if not len(new_data):
            return df, return_check_list
        
        new_data = new_data[['auctionNo' ,'regDate', 'itemId' , 'itemName' , 'count' , 'price' , 'unitPrice']]
        new_data['status'] = 'Saling'
        excecuteDB(new_data)
        return new_data.reset_index(drop=True), return_check_list
    
    res = df_aution_item(target)

    if res.status_code != 200:
        print('Requests Error!!')

    data = res.json()['rows']
    if len(data) == 0:
            return df, return_check_list
    
    new_rows = pd.DataFrame(data)
    new_rows = new_rows[['auctionNo' ,'regDate', 'itemId' , 'itemName' , 'count' , 'price' , 'unitPrice']]
    new_rows['status'] = 'Saling'

    new_items_idx = check_new_items(new_rows, df[df['itemName']==target]) 
    new_items = new_rows.loc[new_items_idx]
   
    if len(new_items) != 0:
        new_items_num = new_items['auctionNo'].values
    else:
        new_items_num = []
    sold_item_idx = check_sold_item(new_rows[~new_rows['auctionNo'].isin(new_items_num)], df[df['itemName']==target])
    if len(sold_item_idx) != 0:
        df['status'].loc[sold_item_idx] = "Saled"
        update_list = df[df['status']=='Saled']
        update_list = update_list[~update_list.index.isin(return_check_list)]
        updateDB(update_list)
        return_check_list += list(sold_item_idx)
        return_check_list = list(set(return_check_list))
    if len(new_items) != 0 :
        print('new items!! / ',target)
        concat_df = pd.concat([df, new_items]).reset_index(drop=True)
        excecuteDB(new_items)
        return concat_df, return_check_list
    else:
        return df, return_check_list

In [11]:
# top_dungon = ['마이스터의 실험실', '파괴된 죽은 자의 성', '노블레스 코드', '노블레스코드:더스크']

# targets = list()

# for keyword in top_dungon:
#     enter_tickets = pd.DataFrame(df_search_item(itemName=keyword).json()['rows'])
#     enter_tickets = enter_tickets[enter_tickets['itemName'].str.contains('입장권')]
#     enter_tickets = enter_tickets[~enter_tickets['itemName'].str.contains('상자')]
#     targets += list(enter_tickets['itemName'].values)
# targets = list(set(targets))
# targets


['노블레스 코드 : 더스크 (익스퍼트) 입장권',
 '파괴된 죽은 자의 성 (노멀) 입장권',
 '노블레스 코드 (노멀) 입장권',
 '파괴된 죽은 자의 성 (익스퍼트) 입장권',
 '파괴된 죽은 자의 성 (마스터) 입장권',
 '마이스터의 실험실 (익스퍼트) 입장권',
 '마이스터의 실험실 (마스터) 입장권',
 '노블레스 코드 : 더스크 (노멀) 입장권',
 '노블레스 코드 : 더스크 (마스터) 입장권',
 '마이스터의 실험실 (노멀) 입장권']

In [ ]:
# start_time = time.time()
df = pd.DataFrame()
saled_check_list = []
while True:
    for target in targets:
        try:
            df, saled_check_list = df_get_auction_reg_item(df,target, saled_check_list)
            flag = False
        except Exception as e:
            print(e)
            time.sleep(2)
    time.sleep(1)

new items!! /  노블레스 코드 : 더스크 (노멀) 입장권
new items!! /  마이스터의 실험실 (익스퍼트) 입장권
new items!! /  노블레스 코드 : 더스크 (마스터) 입장권
new items!! /  파괴된 죽은 자의 성 (익스퍼트) 입장권
new items!! /  노블레스 코드 (익스퍼트) 입장권
new items!! /  파괴된 죽은 자의 성 (마스터) 입장권
new items!! /  노블레스 코드 (노멀) 입장권
new items!! /  마이스터의 실험실 (마스터) 입장권
new items!! /  파괴된 죽은 자의 성 (노멀) 입장권
update!!
1
new items!! /  파괴된 죽은 자의 성 (마스터) 입장권
new items!! /  마이스터의 실험실 (마스터) 입장권
update!!
1
new items!! /  마이스터의 실험실 (마스터) 입장권
update!!
1
update!!
2
new items!! /  마이스터의 실험실 (마스터) 입장권
update!!
4
new items!! /  마이스터의 실험실 (노멀) 입장권
update!!
4
new items!! /  마이스터의 실험실 (마스터) 입장권
new items!! /  노블레스 코드 : 더스크 (마스터) 입장권
update!!
1
new items!! /  파괴된 죽은 자의 성 (마스터) 입장권
new items!! /  노블레스 코드 : 더스크 (마스터) 입장권
update!!
2
new items!! /  노블레스 코드 (익스퍼트) 입장권
new items!! /  마이스터의 실험실 (마스터) 입장권
new items!! /  노블레스 코드 : 더스크 (마스터) 입장권
new items!! /  파괴된 죽은 자의 성 (마스터) 입장권
update!!
1
new items!! /  마이스터의 실험실 (마스터) 입장권
new items!! /  노블레스 코드 (익스퍼트) 입장권
update!!
1
new items!! /  마이스터의 실험실 

update!!
1
new items!! /  마이스터의 실험실 (마스터) 입장권
update!!
2
new items!! /  마이스터의 실험실 (마스터) 입장권
update!!
1
update!!
1
new items!! /  마이스터의 실험실 (익스퍼트) 입장권
new items!! /  노블레스 코드 (익스퍼트) 입장권
new items!! /  파괴된 죽은 자의 성 (노멀) 입장권
update!!
1
new items!! /  노블레스 코드 : 더스크 (마스터) 입장권
new items!! /  마이스터의 실험실 (마스터) 입장권
new items!! /  노블레스 코드 (마스터) 입장권
new items!! /  마이스터의 실험실 (마스터) 입장권
new items!! /  파괴된 죽은 자의 성 (마스터) 입장권
new items!! /  노블레스 코드 (마스터) 입장권
update!!
4
update!!
3
new items!! /  마이스터의 실험실 (마스터) 입장권
new items!! /  파괴된 죽은 자의 성 (익스퍼트) 입장권
new items!! /  마이스터의 실험실 (마스터) 입장권
new items!! /  마이스터의 실험실 (마스터) 입장권
update!!
1
new items!! /  마이스터의 실험실 (마스터) 입장권
new items!! /  노블레스 코드 (익스퍼트) 입장권
new items!! /  마이스터의 실험실 (마스터) 입장권
new items!! /  파괴된 죽은 자의 성 (익스퍼트) 입장권
new items!! /  노블레스 코드 (익스퍼트) 입장권
new items!! /  마이스터의 실험실 (마스터) 입장권
update!!
1
new items!! /  노블레스 코드 (마스터) 입장권
new items!! /  마이스터의 실험실 (익스퍼트) 입장권
update!!
2
new items!! /  마이스터의 실험실 (마스터) 입장권
update!!
1
new items!! /  노블레스 코드 (익스퍼트) 입장권

new items!! /  파괴된 죽은 자의 성 (익스퍼트) 입장권
new items!! /  파괴된 죽은 자의 성 (노멀) 입장권
new items!! /  마이스터의 실험실 (마스터) 입장권
update!!
5
new items!! /  노블레스 코드 : 더스크 (익스퍼트) 입장권
new items!! /  마이스터의 실험실 (마스터) 입장권
update!!
7
new items!! /  파괴된 죽은 자의 성 (마스터) 입장권
new items!! /  노블레스 코드 : 더스크 (익스퍼트) 입장권
update!!
3
new items!! /  마이스터의 실험실 (마스터) 입장권
new items!! /  파괴된 죽은 자의 성 (마스터) 입장권
update!!
2
update!!
4
new items!! /  노블레스 코드 : 더스크 (마스터) 입장권
update!!
1
new items!! /  파괴된 죽은 자의 성 (마스터) 입장권
new items!! /  마이스터의 실험실 (마스터) 입장권
new items!! /  마이스터의 실험실 (마스터) 입장권
new items!! /  마이스터의 실험실 (익스퍼트) 입장권
update!!
1
new items!! /  노블레스 코드 (마스터) 입장권
update!!
4
new items!! /  파괴된 죽은 자의 성 (마스터) 입장권
update!!
1
update!!
1
new items!! /  마이스터의 실험실 (마스터) 입장권
new items!! /  파괴된 죽은 자의 성 (익스퍼트) 입장권
new items!! /  노블레스 코드 : 더스크 (익스퍼트) 입장권
new items!! /  마이스터의 실험실 (마스터) 입장권
update!!
2
new items!! /  노블레스 코드 : 더스크 (익스퍼트) 입장권
new items!! /  노블레스 코드 (노멀) 입장권
update!!
1
new items!! /  마이스터의 실험실 (노멀) 입장권
new items!! /  마이스터의 실험실 (마스터

new items!! /  마이스터의 실험실 (익스퍼트) 입장권
update!!
2
new items!! /  파괴된 죽은 자의 성 (노멀) 입장권
update!!
1
new items!! /  마이스터의 실험실 (마스터) 입장권
new items!! /  마이스터의 실험실 (익스퍼트) 입장권
update!!
2
new items!! /  파괴된 죽은 자의 성 (익스퍼트) 입장권
update!!
3
update!!
5
new items!! /  마이스터의 실험실 (마스터) 입장권
update!!
2
new items!! /  파괴된 죽은 자의 성 (노멀) 입장권
new items!! /  마이스터의 실험실 (마스터) 입장권
new items!! /  노블레스 코드 (익스퍼트) 입장권
new items!! /  마이스터의 실험실 (마스터) 입장권
new items!! /  파괴된 죽은 자의 성 (마스터) 입장권
update!!
2
new items!! /  마이스터의 실험실 (마스터) 입장권
update!!
1
new items!! /  마이스터의 실험실 (마스터) 입장권
new items!! /  노블레스 코드 (마스터) 입장권
new items!! /  마이스터의 실험실 (마스터) 입장권
new items!! /  마이스터의 실험실 (익스퍼트) 입장권
update!!
1
new items!! /  노블레스 코드 : 더스크 (마스터) 입장권
new items!! /  파괴된 죽은 자의 성 (노멀) 입장권
new items!! /  마이스터의 실험실 (마스터) 입장권
update!!
6
new items!! /  파괴된 죽은 자의 성 (마스터) 입장권
new items!! /  노블레스 코드 : 더스크 (노멀) 입장권
update!!
5
update!!
1
new items!! /  마이스터의 실험실 (마스터) 입장권
update!!
3
new items!! /  노블레스 코드 (마스터) 입장권
new items!! /  마이스터의 실험실 (마스터) 입장권
ne

new items!! /  노블레스 코드 (마스터) 입장권
new items!! /  파괴된 죽은 자의 성 (익스퍼트) 입장권
update!!
3
new items!! /  파괴된 죽은 자의 성 (마스터) 입장권
new items!! /  노블레스 코드 (마스터) 입장권
update!!
2
new items!! /  마이스터의 실험실 (마스터) 입장권
new items!! /  마이스터의 실험실 (익스퍼트) 입장권
new items!! /  노블레스 코드 (익스퍼트) 입장권
update!!
3
new items!! /  마이스터의 실험실 (익스퍼트) 입장권
update!!
1
update!!
1
new items!! /  파괴된 죽은 자의 성 (노멀) 입장권
update!!
3
new items!! /  마이스터의 실험실 (마스터) 입장권
new items!! /  마이스터의 실험실 (익스퍼트) 입장권
new items!! /  파괴된 죽은 자의 성 (마스터) 입장권
update!!
1
new items!! /  노블레스 코드 (익스퍼트) 입장권
update!!
1
new items!! /  마이스터의 실험실 (마스터) 입장권
new items!! /  노블레스 코드 (익스퍼트) 입장권
new items!! /  노블레스 코드 : 더스크 (노멀) 입장권
new items!! /  마이스터의 실험실 (마스터) 입장권
update!!
3
update!!
1
new items!! /  마이스터의 실험실 (노멀) 입장권
new items!! /  마이스터의 실험실 (마스터) 입장권
update!!
1
update!!
1
new items!! /  노블레스 코드 : 더스크 (익스퍼트) 입장권
new items!! /  노블레스 코드 (익스퍼트) 입장권
new items!! /  마이스터의 실험실 (마스터) 입장권
new items!! /  마이스터의 실험실 (익스퍼트) 입장권
new items!! /  노블레스 코드 (마스터) 입장권
update!!
2
new item

In [4]:
import  mysql.connector

aws = mysql.connector.Connect(
    database = 'df',
    host = "dungeonandfighter-db.crnv1am9ifjd.ap-northeast-2.rds.amazonaws.com",
    port = 3306,
    user = 'insert',
    password = '1234'
)
cur = aws.cursor()

In [5]:
def excecuteDB(df):
    cols = df.columns
    insert_cols = ",".join(cols)
    for idx, row in df.iterrows():
        insert_value = ['"'+str(row[col])+'"' for col in cols]
        insert_value = ",".join(map(str,insert_value))
        query = 'INSERT INTO ticket_history('+ insert_cols +') VALUES('+ insert_value +');'
        cur.execute(query)
    aws.commit()
    

def updateDB(df):
    if len(df) !=0:
        print("update!!")
        print(len(df))
        
    update_col = "status"
    search_col = "auctionNo"
    for idx, row in df.iterrows():
        update_value = '"' + row['status'] + '"'
        search_value = str(row['auctionNo'])
        query = "UPDATE ticket_history SET " + update_col + "=" + update_value + " WHERE " + search_col + "=" + search_value
        cur.execute(query)
    aws.commit()

In [6]:
query = 'SELECT distinct itemName FROM ticket_history'
cur.execute(query)
targets = cur.fetchall()
targets = [list(t)[0] for t in targets]

In [17]:
aws.close()